In [4]:
import calcbench as cb
from IPython.core.display import display, HTML
import json
import itertools
from tqdm import tqdm, tqdm_notebook
import random
import urllib.error
import requests

In [5]:
disclosure_types = json.loads('[{"arcrole":1100,"name":"Business Description","shortName":"Business Description","tags":null,"longName":"Business Description"},{"arcrole":1110,"name":"Risk Factors","shortName":"Risk Factors","tags":null,"longName":"Risk Factors"},{"arcrole":1120,"name":"Unresolved SEC Comments","shortName":"Unresolved Comments","tags":null,"longName":"Unresolved SEC Comments"},{"arcrole":1200,"name":"Properties","shortName":"Properties","tags":null,"longName":"Properties"},{"arcrole":1300,"name":"Legal Proceedings","shortName":"Legal Proceedings","tags":null,"longName":"Legal Proceedings"},{"arcrole":2410,"name":"Executive Officers","shortName":"Executive Officers","tags":null,"longName":"Executive Officers"},{"arcrole":2300,"name":"Defaults Upon Senior Securities","shortName":"Defaults","tags":null,"longName":"Defaults Upon Senior Securities"},{"arcrole":2500,"name":"Market For Registrants Common Equity","shortName":"Market For Equity","tags":null,"longName":"Market For Registrants Common Equity"},{"arcrole":2600,"name":"Selected Financial Data","shortName":"Selected Data","tags":null,"longName":"Selected Financial Data"},{"arcrole":2700,"name":"Management\u0027s Discussion And Analysis","shortName":"MD\u0026A","tags":null,"longName":"Management\u0027s Discussion And Analysis"},{"arcrole":2710,"name":"Disclosures About Market Risk","shortName":"Market Risk","tags":null,"longName":"Disclosures About Market Risk"},{"arcrole":2810,"name":"Report Of Independent Registered Public Accounting Firm","shortName":"Auditor\u0027s Report","tags":null,"longName":"Report Of Independent Registered Public Accounting Firm"},{"arcrole":2900,"name":"Changes In And Disagreements With Accountants","shortName":"Auditor Changes/Disagreements","tags":null,"longName":"Changes In And Disagreements With Accountants"},{"arcrole":2910,"name":"Controls And Procedures","shortName":"Controls And Procedures","tags":null,"longName":"Controls And Procedures"},{"arcrole":2920,"name":"Other Information","shortName":"Other Information","tags":null,"longName":"Other Information"},{"arcrole":3100,"name":"Corporate Governance","shortName":"Corporate Governance","tags":null,"longName":"Corporate Governance"},{"arcrole":3120,"name":"Security Ownership","shortName":"Security Ownership","tags":null,"longName":"Security Ownership"},{"arcrole":3130,"name":"Relationships","shortName":"Relationships","tags":null,"longName":"Relationships"}]')
disclosure_name_to_disclosure_type = {d['name'] : d['arcrole'] for d in disclosure_types}

In [6]:
periods = [(year, period) for year in range(2010, 2018) for period in [0, 1, 2, 3]]

In [7]:
get_doc_contents = lambda doc : cb.document_contents(blob_id=doc['blob_id'], SEC_ID=doc['sec_filing_id'])

In [29]:
def get_documents_all_companies(disclosure_type, year, period):
    try:
        found_documents = list(cb.document_search(entire_universe=True, document_type=disclosure_type, year=year, period=period))
    except requests.RequestException:
        print(disclosure_type, year, period)
        return
    docs = []
    for  doc in tqdm_notebook(found_documents, desc="{0} {1}-{2}".format(disclosure_type, year, period), leave=False):
        try:            
            docs.append(get_doc_contents(doc))
        except requests.RequestException:
            print(disclosure_type, year, period, doc)

In [9]:
test_doc_periods = list(itertools.product(disclosure_types, periods))

In [10]:
for test_doc, period in tqdm_notebook(test_doc_periods[44:], desc="period loop"):
    get_documents_all_companies(test_doc['arcrole'], year=period[0], period=period[1])

1100 2011 2 {'page_url': 'ftp://ftp.sec.gov/edgar/data/1493040/000105652011000387/nmed10q07312011final.htm', 'local_name': None, 'blob_id': '198050_section110', 'accession_id': 7561, 'received_date': '2011-09-13', 'description': 'New Media Insight Group, Inc. (NMED)', 'fiscal_year': 2012, 'entity_id': 4249, 'sec_filing_id': 198050, 'fiscal_period': '1Q', 'fact_id': 0, 'is_detail': False, 'network_id': 0, 'filing_date': '2011-09-13', 'footnote_type_title': 'Additional 10-K and 10-Q Sections', 'ticker': 'NMED', 'guide_link': None, 'footnote_type': 1100, 'entity_name': 'New Media Insight Group, Inc.', 'document_type': '10-Q', 'SEC_URL': 'http://www.sec.gov/Archives/edgar/data/1493040/000105652011000387/0001056520-11-000387-index.htm'}


1110 2010 3 {'page_url': 'ftp://ftp.sec.gov/edgar/data/26324/000002632410000013/form10q.htm', 'local_name': None, 'blob_id': '198816_section111', 'accession_id': 8327, 'received_date': '2010-11-04', 'description': 'Curtiss Wright Corp (CW)', 'fiscal_year': 2010, 'entity_id': 4650, 'sec_filing_id': 198816, 'fiscal_period': '3Q', 'fact_id': 0, 'is_detail': False, 'network_id': 0, 'filing_date': '2010-11-04', 'footnote_type_title': 'Additional 10-K and 10-Q Sections', 'ticker': 'CW', 'guide_link': None, 'footnote_type': 1110, 'entity_name': 'Curtiss Wright Corp', 'document_type': '10-Q', 'SEC_URL': 'http://www.sec.gov/Archives/edgar/data/26324/000002632410000013/0000026324-10-000013-index.htm'}
1110 2010 3 {'page_url': 'ftp://ftp.sec.gov/edgar/data/1140536/000095012310101712/u09753e10vq.htm', 'local_name': None, 'blob_id': '193887_section111', 'accession_id': 3398, 'received_date': '2010-11-05', 'description': 'Willis Towers Watson Plc (WLTW)', 'fiscal_year': 2010, 'entity_id': 1534, 'sec

1110 2011 2 {'page_url': 'ftp://ftp.sec.gov/edgar/data/1518238/000149473311000044/advancecloudstorage10qjune30.htm', 'local_name': None, 'blob_id': '198317_section111', 'accession_id': 7828, 'received_date': '2011-09-07', 'description': 'Montalvo Spirits, Inc. (ACSRD)', 'fiscal_year': 2012, 'entity_id': 4497, 'sec_filing_id': 198317, 'fiscal_period': '1Q', 'fact_id': 0, 'is_detail': False, 'network_id': 0, 'filing_date': '2011-09-07', 'footnote_type_title': 'Additional 10-K and 10-Q Sections', 'ticker': 'ACSRD', 'guide_link': None, 'footnote_type': 1110, 'entity_name': 'Montalvo Spirits, Inc.', 'document_type': '10-Q', 'SEC_URL': 'http://www.sec.gov/Archives/edgar/data/1518238/000149473311000044/0001494733-11-000044-index.htm'}
1110 2011 2 {'page_url': 'ftp://ftp.sec.gov/edgar/data/1211524/000121152411000193/qjune2011.htm', 'local_name': None, 'blob_id': '194557_section111', 'accession_id': 4068, 'received_date': '2011-08-26', 'description': 'Allied Resources Inc (ALOD)', 'fiscal_year'

1110 2011 2 {'page_url': 'ftp://ftp.sec.gov/edgar/data/1377318/000143209311000544/huntglobal10q063011.htm', 'local_name': None, 'blob_id': '196402_section111', 'accession_id': 5913, 'received_date': '2011-08-15', 'description': 'Hunt Global Resources, Inc. (HGCO)', 'fiscal_year': 2011, 'entity_id': 2976, 'sec_filing_id': 196402, 'fiscal_period': '2Q', 'fact_id': 0, 'is_detail': False, 'network_id': 0, 'filing_date': '2011-08-15', 'footnote_type_title': 'Additional 10-K and 10-Q Sections', 'ticker': 'HGCO', 'guide_link': None, 'footnote_type': 1110, 'entity_name': 'Hunt Global Resources, Inc.', 'document_type': '10-Q', 'SEC_URL': 'http://www.sec.gov/Archives/edgar/data/1377318/000143209311000544/0001432093-11-000544-index.htm'}
1110 2011 2 {'page_url': 'ftp://ftp.sec.gov/edgar/data/1011452/000101145211000020/tenq2a.htm', 'local_name': None, 'blob_id': '190793_section111', 'accession_id': 304, 'received_date': '2011-09-06', 'description': 'Medical Information Technology, Inc. (CIK0001011

1110 2011 2 {'page_url': 'ftp://ftp.sec.gov/edgar/data/1442101/000100233411000346/j11q.htm', 'local_name': None, 'blob_id': '197491_section111', 'accession_id': 7002, 'received_date': '2011-08-17', 'description': 'Western Lucrative Enterprises, Inc. (WLUC)', 'fiscal_year': 2011, 'entity_id': 3786, 'sec_filing_id': 197491, 'fiscal_period': '2Q', 'fact_id': 0, 'is_detail': False, 'network_id': 0, 'filing_date': '2011-08-17', 'footnote_type_title': 'Additional 10-K and 10-Q Sections', 'ticker': 'WLUC', 'guide_link': None, 'footnote_type': 1110, 'entity_name': 'Western Lucrative Enterprises, Inc.', 'document_type': '10-Q', 'SEC_URL': 'http://www.sec.gov/Archives/edgar/data/1442101/000100233411000346/0001002334-11-000346-index.htm'}
1110 2011 2 {'page_url': 'ftp://ftp.sec.gov/edgar/data/1099568/000109956811000005/f10qajune2011.htm', 'local_name': None, 'blob_id': '193112_section111', 'accession_id': 2623, 'received_date': '2011-09-15', 'description': 'Westgate Acquisitions Corp (WESTG)', 'f

ConnectionError: ('Connection aborted.', TimeoutError(10060, 'A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond', None, 10060, None))

In [18]:
for test_doc, period in tqdm_notebook(test_doc_periods[44:], desc="period loop"):
    get_documents_all_companies(test_doc['arcrole'], year=period[0], period=period[1])

1110 2016 2 {'page_url': 'ftp://ftp.sec.gov/edgar/data/1472595/000147259516000220/june302016-form10q.htm', 'local_name': None, 'blob_id': '607039_section111', 'accession_id': 165832, 'received_date': '2016-08-04', 'description': 'R1 Rcm Inc. (ACHI)', 'fiscal_year': 2016, 'entity_id': 4024, 'sec_filing_id': 607039, 'fiscal_period': '2Q', 'fact_id': 0, 'is_detail': False, 'network_id': 0, 'filing_date': '2016-08-04', 'footnote_type_title': 'Additional 10-K and 10-Q Sections', 'ticker': 'ACHI', 'guide_link': None, 'footnote_type': 1110, 'entity_name': 'R1 Rcm Inc.', 'document_type': '10-Q', 'SEC_URL': 'http://www.sec.gov/Archives/edgar/data/1472595/000147259516000220/0001472595-16-000220-index.htm'}
1110 2016 2 {'page_url': 'ftp://ftp.sec.gov/edgar/data/702513/000143774916036550/boch20160630_10q.htm', 'local_name': None, 'blob_id': '608096_section111', 'accession_id': 166230, 'received_date': '2016-08-05', 'description': 'Bank of Commerce Holdings (BOCH)', 'fiscal_year': 2016, 'entity_id'

1110 2016 2 {'page_url': 'ftp://ftp.sec.gov/edgar/data/315374/000114420416122905/v447426_10q.htm', 'local_name': None, 'blob_id': '616164_section111', 'accession_id': 169998, 'received_date': '2016-09-06', 'description': 'Hurco Companies Inc (HURC)', 'fiscal_year': 2016, 'entity_id': 4749, 'sec_filing_id': 616164, 'fiscal_period': '3Q', 'fact_id': 0, 'is_detail': False, 'network_id': 0, 'filing_date': '2016-09-06', 'footnote_type_title': 'Additional 10-K and 10-Q Sections', 'ticker': 'HURC', 'guide_link': None, 'footnote_type': 1110, 'entity_name': 'Hurco Companies Inc', 'document_type': '10-Q', 'SEC_URL': 'http://www.sec.gov/Archives/edgar/data/315374/000114420416122905/0001144204-16-122905-index.htm'}
1110 2016 2 {'page_url': 'ftp://ftp.sec.gov/edgar/data/1495231/000149523116000133/izea16063010q.htm', 'local_name': None, 'blob_id': '611149_section111', 'accession_id': 168015, 'received_date': '2016-08-11', 'description': 'IZEA, Inc. (IZEA)', 'fiscal_year': 2016, 'entity_id': 4269, 's

1110 2016 2 {'page_url': 'ftp://ftp.sec.gov/edgar/data/1321070/000143774916037388/ottw20160630_10q.htm', 'local_name': None, 'blob_id': '612167_section111', 'accession_id': 168626, 'received_date': '2016-08-15', 'description': 'Ottawa Bancorp Inc (OTTW)', 'fiscal_year': 2016, 'entity_id': 2404, 'sec_filing_id': 612167, 'fiscal_period': '2Q', 'fact_id': 0, 'is_detail': False, 'network_id': 0, 'filing_date': '2016-08-15', 'footnote_type_title': 'Additional 10-K and 10-Q Sections', 'ticker': 'OTTW', 'guide_link': None, 'footnote_type': 1110, 'entity_name': 'Ottawa Bancorp Inc', 'document_type': '10-Q', 'SEC_URL': 'http://www.sec.gov/Archives/edgar/data/1321070/000143774916037388/0001437749-16-037388-index.htm'}
1110 2016 2 {'page_url': 'ftp://ftp.sec.gov/edgar/data/1378946/000119312516676617/d221059d10q.htm', 'local_name': None, 'blob_id': '609839_section111', 'accession_id': 167272, 'received_date': '2016-08-09', 'description': "People's United Financial, Inc. (PBCT)", 'fiscal_year': 201

1110 2016 2 {'page_url': 'ftp://ftp.sec.gov/edgar/data/1137547/000162828016018498/ubfo-20160630xq2.htm', 'local_name': None, 'blob_id': '607016_section111', 'accession_id': 165809, 'received_date': '2016-08-04', 'description': 'United Security Bancshares (UBFO)', 'fiscal_year': 2016, 'entity_id': 1503, 'sec_filing_id': 607016, 'fiscal_period': '2Q', 'fact_id': 0, 'is_detail': False, 'network_id': 0, 'filing_date': '2016-08-04', 'footnote_type_title': 'Additional 10-K and 10-Q Sections', 'ticker': 'UBFO', 'guide_link': None, 'footnote_type': 1110, 'entity_name': 'United Security Bancshares', 'document_type': '10-Q', 'SEC_URL': 'http://www.sec.gov/Archives/edgar/data/1137547/000162828016018498/0001628280-16-018498-index.htm'}
1110 2016 2 {'page_url': 'ftp://ftp.sec.gov/edgar/data/1052100/000155837016006946/wdr-20160630x10q.htm', 'local_name': None, 'blob_id': '603911_section111', 'accession_id': 164753, 'received_date': '2016-07-29', 'description': 'Waddell & Reed Financial Inc (WDR)', '

1110 2016 3 {'page_url': 'ftp://ftp.sec.gov/edgar/data/1317630/000162828016020815/itc930201610q.htm', 'local_name': None, 'blob_id': '626844_section111', 'accession_id': 172789, 'received_date': '2016-11-04', 'description': 'ITC Holdings Corp. (ITC)', 'fiscal_year': 2016, 'entity_id': 2373, 'sec_filing_id': 626844, 'fiscal_period': '3Q', 'fact_id': 0, 'is_detail': False, 'network_id': 0, 'filing_date': '2016-11-04', 'footnote_type_title': 'Additional 10-K and 10-Q Sections', 'ticker': 'ITC', 'guide_link': None, 'footnote_type': 1110, 'entity_name': 'ITC Holdings Corp.', 'document_type': '10-Q', 'SEC_URL': 'http://www.sec.gov/Archives/edgar/data/1317630/000162828016020815/0001628280-16-020815-index.htm'}
1110 2016 3 {'page_url': 'ftp://ftp.sec.gov/edgar/data/350868/000110465916156308/a16-17251_110q.htm', 'local_name': None, 'blob_id': '630674_section111', 'accession_id': 174994, 'received_date': '2016-11-10', 'description': 'Iteris, Inc. (ITI)', 'fiscal_year': 2017, 'entity_id': 4839, '

...

2710 2016 2 {'page_url': 'ftp://ftp.sec.gov/edgar/data/1472595/000147259516000220/june302016-form10q.htm', 'local_name': None, 'blob_id': '607039_section271', 'accession_id': 165832, 'received_date': '2016-08-04', 'description': 'R1 Rcm Inc. (ACHI)', 'fiscal_year': 2016, 'entity_id': 4024, 'sec_filing_id': 607039, 'fiscal_period': '2Q', 'fact_id': 0, 'is_detail': False, 'network_id': 0, 'filing_date': '2016-08-04', 'footnote_type_title': 'Additional 10-K and 10-Q Sections', 'ticker': 'ACHI', 'guide_link': None, 'footnote_type': 2710, 'entity_name': 'R1 Rcm Inc.', 'document_type': '10-Q', 'SEC_URL': 'http://www.sec.gov/Archives/edgar/data/1472595/000147259516000220/0001472595-16-000220-index.htm'}
2710 2016 2 {'page_url': 'ftp://ftp.sec.gov/edgar/data/7789/000000778916000069/asb0630201610q.htm', 'local_name': None, 'blob_id': '603524_section271', 'accession_id': 164658, 'received_date': '2016-07-28', 'description': 'Associated Banc-Corp (ASB)', 'fiscal_year': 2016, 'entity_id': 5663, 's

2710 2016 2 {'page_url': 'ftp://ftp.sec.gov/edgar/data/315374/000114420416122905/v447426_10q.htm', 'local_name': None, 'blob_id': '616164_section271', 'accession_id': 169998, 'received_date': '2016-09-06', 'description': 'Hurco Companies Inc (HURC)', 'fiscal_year': 2016, 'entity_id': 4749, 'sec_filing_id': 616164, 'fiscal_period': '3Q', 'fact_id': 0, 'is_detail': False, 'network_id': 0, 'filing_date': '2016-09-06', 'footnote_type_title': 'Additional 10-K and 10-Q Sections', 'ticker': 'HURC', 'guide_link': None, 'footnote_type': 2710, 'entity_name': 'Hurco Companies Inc', 'document_type': '10-Q', 'SEC_URL': 'http://www.sec.gov/Archives/edgar/data/315374/000114420416122905/0001144204-16-122905-index.htm'}
2710 2016 2 {'page_url': 'ftp://ftp.sec.gov/edgar/data/1495231/000149523116000133/izea16063010q.htm', 'local_name': None, 'blob_id': '611149_section271', 'accession_id': 168015, 'received_date': '2016-08-11', 'description': 'IZEA, Inc. (IZEA)', 'fiscal_year': 2016, 'entity_id': 4269, 's

2710 2016 2 {'page_url': 'ftp://ftp.sec.gov/edgar/data/1097792/000121390016016316/f10q0616_omintoinc.htm', 'local_name': None, 'blob_id': '614161_section271', 'accession_id': 169493, 'received_date': '2016-08-22', 'description': 'Ominto, Inc. (OMNT)', 'fiscal_year': 2016, 'entity_id': 1131, 'sec_filing_id': 614161, 'fiscal_period': '3Q', 'fact_id': 0, 'is_detail': False, 'network_id': 0, 'filing_date': '2016-08-22', 'footnote_type_title': 'Additional 10-K and 10-Q Sections', 'ticker': 'OMNT', 'guide_link': None, 'footnote_type': 2710, 'entity_name': 'Ominto, Inc.', 'document_type': '10-Q', 'SEC_URL': 'http://www.sec.gov/Archives/edgar/data/1097792/000121390016016316/0001213900-16-016316-index.htm'}
2710 2016 2 {'page_url': 'ftp://ftp.sec.gov/edgar/data/1321070/000143774916037388/ottw20160630_10q.htm', 'local_name': None, 'blob_id': '612167_section271', 'accession_id': 168626, 'received_date': '2016-08-15', 'description': 'Ottawa Bancorp Inc (OTTW)', 'fiscal_year': 2016, 'entity_id': 24

2710 2016 2 {'page_url': 'ftp://ftp.sec.gov/edgar/data/1052100/000155837016006946/wdr-20160630x10q.htm', 'local_name': None, 'blob_id': '603911_section271', 'accession_id': 164753, 'received_date': '2016-07-29', 'description': 'Waddell & Reed Financial Inc (WDR)', 'fiscal_year': 2016, 'entity_id': 585, 'sec_filing_id': 603911, 'fiscal_period': '2Q', 'fact_id': 0, 'is_detail': False, 'network_id': 0, 'filing_date': '2016-07-29', 'footnote_type_title': 'Additional 10-K and 10-Q Sections', 'ticker': 'WDR', 'guide_link': None, 'footnote_type': 2710, 'entity_name': 'Waddell & Reed Financial Inc', 'document_type': '10-Q', 'SEC_URL': 'http://www.sec.gov/Archives/edgar/data/1052100/000155837016006946/0001558370-16-006946-index.htm'}
2710 2016 2 {'page_url': 'ftp://ftp.sec.gov/edgar/data/801337/000080133716000223/wbs-06302016x10q.htm', 'local_name': None, 'blob_id': '609849_section271', 'accession_id': 167282, 'received_date': '2016-08-09', 'description': 'Webster Financial Corp (WBS)', 'fiscal

ConnectionError: ('Connection aborted.', TimeoutError(10060, 'A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond', None, 10060, None))

In [ ]:
for test_doc, period in tqdm_notebook(test_doc_periods[347:], desc="period loop"):
    get_documents_all_companies(test_doc['arcrole'], year=period[0], period=period[1])

2810 2010 0 {'page_url': 'ftp://ftp.sec.gov/edgar/data/1195933/000119312511047183/d10k.htm', 'local_name': None, 'blob_id': '194506_section281', 'accession_id': 4017, 'received_date': '2011-02-25', 'description': 'Infinity Property & Casualty Corp (IPCC)', 'fiscal_year': 2010, 'entity_id': 1836, 'sec_filing_id': 194506, 'fiscal_period': 'Y', 'fact_id': 0, 'is_detail': False, 'network_id': 0, 'filing_date': '2011-02-25', 'footnote_type_title': 'Additional 10-K and 10-Q Sections', 'ticker': 'IPCC', 'guide_link': None, 'footnote_type': 2810, 'entity_name': 'Infinity Property & Casualty Corp', 'document_type': '10-K', 'SEC_URL': 'http://www.sec.gov/Archives/edgar/data/1195933/000119312511047183/0001193125-11-047183-index.htm'}


2810 2011 0 {'page_url': 'ftp://ftp.sec.gov/edgar/data/1420488/000118811211003510/t72163_10k.htm', 'local_name': None, 'blob_id': '213422_section281', 'accession_id': 23026, 'received_date': '2011-12-20', 'description': 'Malvern Federal Bancorp Inc (CIK0001420488)', 'fiscal_year': 2011, 'entity_id': 3526, 'sec_filing_id': 213422, 'fiscal_period': 'Y', 'fact_id': 0, 'is_detail': False, 'network_id': 0, 'filing_date': '2011-12-20', 'footnote_type_title': 'Additional 10-K and 10-Q Sections', 'ticker': 'CIK0001420488', 'guide_link': None, 'footnote_type': 2810, 'entity_name': 'Malvern Federal Bancorp Inc', 'document_type': '10-K', 'SEC_URL': 'http://www.sec.gov/Archives/edgar/data/1420488/000118811211003510/0001188112-11-003510-index.htm'}
2810 2011 0 {'page_url': 'ftp://ftp.sec.gov/edgar/data/1122976/000119312512067181/d263168d10k.htm', 'local_name': None, 'blob_id': '215741_section281', 'accession_id': 25345, 'received_date': '2012-02-17', 'description': 'Polyone Corp (POL)', 'fiscal_yea

2810 2011 3 {'page_url': 'ftp://ftp.sec.gov/edgar/data/1420488/000118811211003510/t72163_10k.htm', 'local_name': None, 'blob_id': '213422_section281', 'accession_id': 23026, 'received_date': '2011-12-20', 'description': 'Malvern Federal Bancorp Inc (CIK0001420488)', 'fiscal_year': 2011, 'entity_id': 3526, 'sec_filing_id': 213422, 'fiscal_period': 'Y', 'fact_id': 0, 'is_detail': False, 'network_id': 0, 'filing_date': '2011-12-20', 'footnote_type_title': 'Additional 10-K and 10-Q Sections', 'ticker': 'CIK0001420488', 'guide_link': None, 'footnote_type': 2810, 'entity_name': 'Malvern Federal Bancorp Inc', 'document_type': '10-K', 'SEC_URL': 'http://www.sec.gov/Archives/edgar/data/1420488/000118811211003510/0001188112-11-003510-index.htm'}


2810 2012 0 {'page_url': 'ftp://ftp.sec.gov/edgar/data/906448/000109690613000398/vu110k.htm', 'local_name': None, 'blob_id': '252968_section281', 'accession_id': 63254, 'received_date': '2013-03-28', 'description': 'Vu1 CORP (VUOC)', 'fiscal_year': 2012, 'entity_id': 6708, 'sec_filing_id': 252968, 'fiscal_period': 'Y', 'fact_id': 0, 'is_detail': False, 'network_id': 0, 'filing_date': '2013-03-28', 'footnote_type_title': 'Additional 10-K and 10-Q Sections', 'ticker': 'VUOC', 'guide_link': None, 'footnote_type': 2810, 'entity_name': 'Vu1 CORP', 'document_type': '10-K', 'SEC_URL': 'http://www.sec.gov/Archives/edgar/data/906448/000109690613000398/0001096906-13-000398-index.htm'}


294/|/ 68%|| 294/435 [00:41<00:19,  7.09it/s]                                                                          2810 2012 2 {'page_url': 'ftp://ftp.sec.gov/edgar/data/723531/000119312512309041/d336374d10k.htm', 'local_name': None, 'blob_id': '230836_section281', 'accession_id': 41122, 'received_date': '2012-07-20', 'description': 'Paychex Inc (PAYX)', 'fiscal_year': 2012, 'entity_id': 5352, 'sec_filing_id': 230836, 'fiscal_period': 'Y', 'fact_id': 0, 'is_detail': False, 'network_id': 0, 'filing_date': '2012-07-20', 'footnote_type_title': 'Additional 10-K and 10-Q Sections', 'ticker': 'PAYX', 'guide_link': None, 'footnote_type': 2810, 'entity_name': 'Paychex Inc', 'document_type': '10-K', 'SEC_URL': 'http://www.sec.gov/Archives/edgar/data/723531/000119312512309041/0001193125-12-309041-index.htm'}


2910 2011 1 {'page_url': 'ftp://ftp.sec.gov/edgar/data/779152/000092623611000032/jkhy2011q3.htm', 'local_name': None, 'blob_id': '201855_section291', 'accession_id': 11367, 'received_date': '2011-05-06', 'description': 'Henry Jack & Associates Inc (JKHY)', 'fiscal_year': 2011, 'entity_id': 5667, 'sec_filing_id': 201855, 'fiscal_period': '3Q', 'fact_id': 0, 'is_detail': False, 'network_id': 0, 'filing_date': '2011-05-06', 'footnote_type_title': 'Additional 10-K and 10-Q Sections', 'ticker': 'JKHY', 'guide_link': None, 'footnote_type': 2910, 'entity_name': 'Henry Jack & Associates Inc', 'document_type': '10-Q', 'SEC_URL': 'http://www.sec.gov/Archives/edgar/data/779152/000092623611000032/0000926236-11-000032-index.htm'}


2910 2011 2 {'page_url': 'ftp://ftp.sec.gov/edgar/data/1096298/000121152411000217/qjune2011.htm', 'local_name': None, 'blob_id': '193020_section291', 'accession_id': 2531, 'received_date': '2011-08-31', 'description': 'Asia8, Inc. (CIK0001096298)', 'fiscal_year': 2011, 'entity_id': 1106, 'sec_filing_id': 193020, 'fiscal_period': '2Q', 'fact_id': 0, 'is_detail': False, 'network_id': 0, 'filing_date': '2011-08-31', 'footnote_type_title': 'Additional 10-K and 10-Q Sections', 'ticker': 'CIK0001096298', 'guide_link': None, 'footnote_type': 2910, 'entity_name': 'Asia8, Inc.', 'document_type': '10-Q/A', 'SEC_URL': 'http://www.sec.gov/Archives/edgar/data/1096298/000121152411000217/0001211524-11-000217-index.htm'}
2910 2011 2 {'page_url': 'ftp://ftp.sec.gov/edgar/data/715812/000071581211000005/form10q.htm', 'local_name': None, 'blob_id': '200869_section291', 'accession_id': 10381, 'received_date': '2011-09-15', 'description': 'Biosynergy Inc (BSYN)', 'fiscal_year': 2012, 'entity_id': 5277, 'sec

2910 2011 2 {'page_url': 'ftp://ftp.sec.gov/edgar/data/1377318/000143209311000544/huntglobal10q063011.htm', 'local_name': None, 'blob_id': '196402_section291', 'accession_id': 5913, 'received_date': '2011-08-15', 'description': 'Hunt Global Resources, Inc. (HGCO)', 'fiscal_year': 2011, 'entity_id': 2976, 'sec_filing_id': 196402, 'fiscal_period': '2Q', 'fact_id': 0, 'is_detail': False, 'network_id': 0, 'filing_date': '2011-08-15', 'footnote_type_title': 'Additional 10-K and 10-Q Sections', 'ticker': 'HGCO', 'guide_link': None, 'footnote_type': 2910, 'entity_name': 'Hunt Global Resources, Inc.', 'document_type': '10-Q', 'SEC_URL': 'http://www.sec.gov/Archives/edgar/data/1377318/000143209311000544/0001432093-11-000544-index.htm'}
2910 2011 2 {'page_url': 'ftp://ftp.sec.gov/edgar/data/1095133/000109513311000027/isa3qtr201110qver75.htm', 'local_name': None, 'blob_id': '192993_section291', 'accession_id': 2504, 'received_date': '2011-09-15', 'description': 'Isa Internationale Inc (ISATOB)', 

2910 2011 2 {'page_url': 'ftp://ftp.sec.gov/edgar/data/790179/000102317511000433/f10qv2edgar.htm', 'local_name': None, 'blob_id': '202034_section291', 'accession_id': 11546, 'received_date': '2011-08-11', 'description': 'Prestige Capital Corp (PGEC)', 'fiscal_year': 2011, 'entity_id': 5732, 'sec_filing_id': 202034, 'fiscal_period': '2Q', 'fact_id': 0, 'is_detail': False, 'network_id': 0, 'filing_date': '2011-08-11', 'footnote_type_title': 'Additional 10-K and 10-Q Sections', 'ticker': 'PGEC', 'guide_link': None, 'footnote_type': 2910, 'entity_name': 'Prestige Capital Corp', 'document_type': '10-Q/A', 'SEC_URL': 'http://www.sec.gov/Archives/edgar/data/790179/000102317511000433/0001023175-11-000433-index.htm'}
2910 2011 2 {'page_url': 'ftp://ftp.sec.gov/edgar/data/1329606/000150441211000059/probeq22011.htm', 'local_name': None, 'blob_id': '195657_section291', 'accession_id': 5168, 'received_date': '2011-08-15', 'description': 'Probe Manufacturing Inc (CETY)', 'fiscal_year': 2011, 'entity

2910 2011 3 {'page_url': 'ftp://ftp.sec.gov/edgar/data/1510961/000148725211000074/ucv1septq.htm', 'local_name': None, 'blob_id': '198255_section291', 'accession_id': 7766, 'received_date': '2011-10-07', 'description': 'USChina Venture I Inc (CIK0001510961)', 'fiscal_year': 2011, 'entity_id': 4437, 'sec_filing_id': 198255, 'fiscal_period': '3Q', 'fact_id': 0, 'is_detail': False, 'network_id': 0, 'filing_date': '2011-10-07', 'footnote_type_title': 'Additional 10-K and 10-Q Sections', 'ticker': 'CIK0001510961', 'guide_link': None, 'footnote_type': 2910, 'entity_name': 'USChina Venture I Inc', 'document_type': '10-Q', 'SEC_URL': 'http://www.sec.gov/Archives/edgar/data/1510961/000148725211000074/0001487252-11-000074-index.htm'}
2910 2011 3 {'page_url': 'ftp://ftp.sec.gov/edgar/data/1510962/000148725211000078/ucv1septq.htm', 'local_name': None, 'blob_id': '198257_section291', 'accession_id': 7768, 'received_date': '2011-10-07', 'description': 'USChina Venture II Inc (CIK0001510962)', 'fiscal